Name: Aylin Rasteh

Student ID: 400104964

In this assignment, we will implement and test REINFORCE and PPO, which are both on-policy RL algortihms.

# REINFORCE algorithm **(40 points)**

## Setup

We must first install the required packages.

In [1]:
!pip -q install gymnasium[mujoco]
!pip install imageio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 22.7 MB/s eta 0:00:00


In [2]:
import gymnasium as gym
import random
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from collections import namedtuple, deque
import imageio

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions import Categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Explore the environment

We will train an REINFORCE agent on the `CartPole` environment.

This code displays a video given it's path.

In [3]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Explore the `CartPole` environment using random actions. At each timestep, render the current frame, and use it to make a video of the trajectory.

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
frames = []

env.reset()
for _ in range(100):
    frames.append(env.render())
    # select a random action
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./CartPole.mp4', frames, fps=25)
show_video('./CartPole.mp4')


## Policy Network **(10 points)**

Complete the following code to build an agent that predicts the the probability of playing each action, given the state.

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        # Define the Policy Network architecture
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # predict the probability of playing each action
        x = F.relu(self.fc1(x))
        action_probs = F.softmax(self.fc2(x), dim=-1)
        return action_probs

## Agent **(20 points)**

REINFORCE algorithm works by interacting with an environment by taking actions based on a policy. As the agent collects rewards from the environment, it records the outcomes and the **log probabilities** of the actions it took. At the end of an episode, the algorithm calculates the total **discounted reward** from each step—this is known as the return.

$$ R_t = \sum_{k=t}^{T} \gamma^{k-t} r_k
 $$

These returns are used to weight the logged probabilities, actions that lead to higher returns are made more probable.


$$ \theta \leftarrow \theta + \alpha \sum_{t=0}^{T-1} \gamma^t R_t \nabla_\theta \log \pi_\theta(a_t|s_t)
 $$


In [ ]:
class REINFORCEAgent:
    def __init__(self, policy, optimizer, gamma=0.99):
        self.policy = policy
        self.optimizer = optimizer
        self.gamma = gamma
        self.log_probs = []
        self.rewards = []

    def select_action(self, state):
        # select an action by sampling from the actor's response
        state = torch.from_numpy(state).float().unsqueeze(0)
        action_probs = self.policy(state)
        dist = torch.distributions.Categorical(action_probs)
        action = dist.sample()
        self.log_probs.append(dist.log_prob(action))
        # print(self.log_probs[-1])
        return action.item()

    def update_policy(self):
        R = 0
        policy_loss = []
        returns = []

        # Calculate the discounted reward
        for r in reversed(self.rewards):
            R = r + self.gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns)
        if returns.std() > 0:
            returns = (returns - returns.mean()) / returns.std()
        else:
            returns = returns - returns.mean()

        # Calculate the policy loss
        for log_prob, R in zip(self.log_probs, returns):
            policy_loss.append(-log_prob * R)

        self.optimizer.zero_grad()
        policy_loss = torch.cat(policy_loss).sum()
        policy_loss.backward()
        self.optimizer.step()

        # Reset the rewards and log probabilities
        del self.rewards[:]
        del self.log_probs[:]

    def store_reward(self, reward):
        self.rewards.append(reward)

## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [ ]:
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
lr = 0.01

policy = PolicyNetwork(input_size, 128, output_size)
optimizer = optim.Adam(policy.parameters(), lr=lr)
agent = REINFORCEAgent(policy, optimizer)

num_episodes = 1000

for episode in range(num_episodes):
    state, info = env.reset()
    total_reward = 0

    # collect rewards and log probabilities for updating the policy in a loop
    while True:
        action = agent.select_action(state)
        state, reward, done, truncated, info = env.step(action)
        agent.store_reward(reward)
        total_reward += reward
        if done or truncated:
            break

    agent.update_policy()
    if episode % 50 == 0:
        print(f'Episode {episode+1}: Total Reward = {total_reward}')
env.close()

Episode 1: Total Reward = 12.0
Episode 51: Total Reward = 75.0
Episode 101: Total Reward = 88.0
Episode 151: Total Reward = 281.0
Episode 201: Total Reward = 500.0
Episode 251: Total Reward = 278.0
Episode 301: Total Reward = 500.0
Episode 351: Total Reward = 500.0
Episode 401: Total Reward = 500.0
Episode 451: Total Reward = 500.0
Episode 501: Total Reward = 500.0
Episode 551: Total Reward = 500.0
Episode 601: Total Reward = 92.0
Episode 651: Total Reward = 101.0
Episode 701: Total Reward = 500.0
Episode 751: Total Reward = 500.0
Episode 801: Total Reward = 500.0
Episode 851: Total Reward = 500.0
Episode 901: Total Reward = 500.0
Episode 951: Total Reward = 500.0


## Evaluation **(5 points)**

Here we use the trained agent and collect a trajectory using it's policy. Calculate the cumulative reward by adding rewards in each time space. Save and display the video of this run in the end.

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# run the policy in the environment in a loop
while True:
    action = agent.select_action(state)
    state, reward, done, truncated, info = env.step(action)
    frames.append(env.render())
    total_reward += reward
    if done or truncated:
        break

env.close()
print(f'Total Reward: {total_reward}')

imageio.mimsave('./eval_reinforce.mp4', frames, fps=25)
show_video('./eval_reinforce.mp4')

Total Reward: 500.0


# Proximal Policy Optimization **(60 points)**

## Setup

## Explore the environment

This code is essential for rendering MUJOCO based environments.

In [4]:
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

env: MUJOCO_GL=egl


We will train a PPO agent in the `HalfCheetah` environment. This environment features continuous actions and more complex mechanics.

Explore this environment using random actions as well, and display the video of the resulting trajectory.

* What are the observation and action spaces of this environment?

* Are values bounded?

In [5]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
env.reset()
frames = []

for _ in range(100):
    frames.append(env.render())
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./HalfCheetah.mp4', frames, fps=25)
show_video('./HalfCheetah.mp4')


## Actor & Critic **(15 points)**

Proximal Policy Optimization (PPO) is an advanced reinforcement learning algorithm that uses separate actor and critic networks to optimize policy performance.

The actor network is responsible for predicting a probability distribution over actions (discrete) or estimating the value for each action (continuous), given the current state, while the critic network evaluates how good the action taken by the actor is, by predicting the reward based on state.


In [6]:
class Actor(nn.Module):
    def __init__(self, state_dim, hidden_size, action_dim):
        super(Actor, self).__init__()
        # Define the Actor architecture
        # self.shared_layers = nn.Sequential(
        # nn.Linear(state_dim, hidden_size)
        # self.fc1 = nn.Linear(state_dim, hidden_size)
        # # self.fc2 = nn.Linear(hidden_size, hidden_size)
        # # self.mu_head = nn.Linear(hidden_size, action_dim)
        # # self.std_head = nn.Linear(hidden_size, action_dim)
        # # nn.Linear(state_dim, hidden_size),
        # # nn.LeakyReLU(),
        # # nn.Linear(hidden_size, hidden_size),
        # # nn.LayerNorm(hidden_size),
        # # nn.LeakyReLU()
        # )
        # self.mu_head = nn.Linear(hidden_size, action_dim)
        # self.std_head = nn.Linear(hidden_size, action_dim)
        self.shared_layers = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.LeakyReLU()
        )
        self.mu_head = nn.Linear(hidden_size, action_dim)
        self.log_std = nn.Parameter(torch.ones(action_dim) * 0)

    def forward(self, state):
        # In case of continuous environment, we usually
        # predict a mean and std for each action and sample
        # the action from a normal distribution
        # x = F.relu(self.fc1(state))
        # x = F.relu(self.fc2(x))
        # mu = F.tanh(self.mu_head(x))
        # std = torch.exp(self.std_head(x))
        # return mu, std
        x = self.shared_layers(state)
        mu = F.tanh(self.mu_head(x))
        std = F.softplus(self.log_std)
        return mu, std

class Critic(nn.Module):
    def __init__(self, state_dim, hidden_size):
        super(Critic, self).__init__()
        # Define the Critic architecture
        self.fc1 = nn.Linear(state_dim, hidden_size)
        # self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.value_head = nn.Linear(hidden_size, 1)

    def forward(self, state):
        # Predict the value of state
        x = F.relu(self.fc1(state))
        # x = F.relu(self.fc2(x))
        value = self.value_head(x)
        return value


## Memory

PPO algorithms need to store sequences of actions, states, log probabilities, rewards, and state values to train the agent. This data is captured in the `Memory` class, which facilitates batch processing by holding and then clearing these elements at the end of each training iteration.

In [7]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]


## Agent **(35 points)**

In PPO, the actor's goal is to maximize the expected return. However, direct maximization can cause large policy updates, risking instability. To prevent this, PPO employs a clipping mechanism, limiting policy changes to a defined range.

$$ L^{CLIP}(\theta) = \hat{\mathbb{E}}_t \left[ \min(r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t) \right]
 $$

Additionally, it uses a probability ratio to scale updates, ensuring changes This ratio provides a scaling factor for the policy updates, ensuring that changes are made in proportion to the improvement in policy performance.

$$ r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}
 $$

 The critic aims to minimize the error between its predictions and the actual returns.

 $$ L^{VF}(\phi) = \left( V_\phi(s_t) - \hat{R}_t \right)^2
 $$

In [8]:
class PPO(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=64, lr=1e-4, gamma=0.99, epochs=4, eps_clip=0.2):
        super(PPO, self).__init__()
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.epochs = epochs

        self.actor = Actor(state_dim, hidden_size, action_dim)
        self.critic = Critic(state_dim, hidden_size)

        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=lr)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=lr)
        self.memory = Memory()

    def select_action(self, state):
        # Save state, action, log probability and state value of current step in the memory buffer.
        # predict the actions by sampling from a normal distribution
        # based on the mean and std calculated by actor
        state = torch.tensor(state, dtype=torch.float32)
        mu, std = self.actor(state)
        dist = torch.distributions.Normal(mu, std)
        action = dist.sample()
        action_logprob = dist.log_prob(action)


        self.memory.states.append(state)
        self.memory.actions.append(action)
        self.memory.logprobs.append(action_logprob)

        value = self.critic(state)
        self.memory.state_values.append(value)

        return action.detach().numpy()

    def evaluate(self, state, action):
        # evaluate the state value of this state and log probability of choosing this action

        mu, std = self.actor(state)
        dist = torch.distributions.Normal(mu, std)
        action_logprobs = dist.log_prob(action)
        entropy = dist.entropy()
        state_value = self.critic(state)

        return action_logprobs, state_value, entropy

    def update(self):
        rewards = []
        discounted_reward = 0
        # Calculate discounted rewards
        for reward in reversed(self.memory.rewards):
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        if rewards.std() > 0:
            rewards = (rewards - rewards.mean()) / rewards.std()
        else:
            rewards = rewards - rewards.mean()

        # load saved states, actions, log probs, and state values
        old_states = torch.stack(self.memory.states)
        old_actions = torch.stack(self.memory.actions)
        old_logprobs = torch.stack(self.memory.logprobs)
        old_rewards = torch.tensor(self.memory.rewards, dtype=torch.float32)
        old_state_values = torch.stack(self.memory.state_values).squeeze()

        # Calculate advantages for each timestep (usually difference of rewards and state values)
        advantages = rewards - old_state_values.detach()
        advantages = advantages.unsqueeze(-1)

        loss_ac = 0
        loss_cri = 0
        for _ in range(self.epochs):
            # calculate logprobs and state values based on the new policy
            logprobs, state_values, entropy = self.evaluate(old_states, old_actions)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Calculate the loss function and perform the optimization
            # print(ratios.shape, advantages.shape, logprobs.shape)
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss_actor = (-torch.min(surr1, surr2).mean() - 0.    01 * entropy).mean()
            # loss_actor = -torch.min(surr1, surr2).mean().mean()
            loss_critic = F.mse_loss(rewards, torch.squeeze(state_values))

            self.optimizer_actor.zero_grad()
            loss_actor.backward()
            loss_ac += loss_actor.item()
            self.optimizer_actor.step()

            self.optimizer_critic.zero_grad()
            loss_critic.backward()
            loss_cri += loss_critic.item()
            self.optimizer_critic.step()

        # clear the buffer
        self.memory.clear()
        return loss_ac, loss_cri

## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [9]:
env = gym.make("HalfCheetah-v4")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
hidden_size = 64
lr = 0.003

agent = PPO(state_dim, action_dim, hidden_size=hidden_size, lr=lr)

# We need to train for many more steps to achieve acceptable results compared to the last environment
num_episodes = 300

actor_losses = []
critic_losses = []

In [10]:
num_episodes = 2000
moving_rewards = np.array([])
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    # write the training loop
    done = False
    while not done:
        action = agent.select_action(state)
        next_state, reward, done, truncated, _ = env.step(action)
        agent.memory.rewards.append(reward)
        total_reward += reward
        state = next_state
        if done or truncated:
            break
    if episode % 5 == 0:
        loss_ac, loss_cri = agent.update()
        actor_losses.append(loss_ac)
        critic_losses.append(loss_cri)
    moving_rewards = np.append(moving_rewards, total_reward)
    if episode % 20 == 0:
        print(f"actor loss:\t{loss_ac:.6f}")
        print(f"critic loss:\t{loss_cri:.6f}")
        print(f'Episode {episode}: Going Reward = {moving_rewards.mean():.1f}: Std = {moving_rewards.std():.1f}')
        moving_rewards = np.array([])
env.close()

actor loss:	0.962147
critic loss:	5.260053
Episode 0: Going Reward = -366.9: Std = 0.0
actor loss:	-0.116535
critic loss:	4.055648
Episode 20: Going Reward = -363.1: Std = 44.5
actor loss:	-0.005035
critic loss:	3.908350
Episode 40: Going Reward = -287.8: Std = 58.9
actor loss:	0.232269
critic loss:	3.732002
Episode 60: Going Reward = -194.1: Std = 86.0
actor loss:	-0.165133
critic loss:	3.435681
Episode 80: Going Reward = -169.4: Std = 104.0
actor loss:	0.187570
critic loss:	3.849915
Episode 100: Going Reward = -139.3: Std = 93.4
actor loss:	-0.309459
critic loss:	3.717699
Episode 120: Going Reward = -169.5: Std = 139.1
actor loss:	0.185386
critic loss:	3.763195
Episode 140: Going Reward = -32.7: Std = 59.0
actor loss:	0.070712
critic loss:	3.630262
Episode 160: Going Reward = -12.8: Std = 61.9
actor loss:	-0.374099
critic loss:	3.123634
Episode 180: Going Reward = -36.2: Std = 215.9
actor loss:	0.488369
critic loss:	3.793568
Episode 200: Going Reward = 42.8: Std = 58.5
actor loss:	0.

In [15]:
num_episodes = 1000
moving_rewards = np.array([])
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    # write the training loop
    done = False
    while not done:
        action = agent.select_action(state)
        next_state, reward, done, truncated, _ = env.step(action)
        agent.memory.rewards.append(reward)
        total_reward += reward
        state = next_state
        if done or truncated:
            break
    if episode % 5 == 0:
        loss_ac, loss_cri = agent.update()
        actor_losses.append(loss_ac)
        critic_losses.append(loss_cri)
    moving_rewards = np.append(moving_rewards, total_reward)
    if episode % 20 == 0:
        print(f"actor loss:\t{loss_ac:.6f}")
        print(f"critic loss:\t{loss_cri:.6f}")
        print(f'Episode {episode}: Going Reward = {moving_rewards.mean():.1f}: Std = {moving_rewards.std():.1f}')
        moving_rewards = np.array([])
env.close()

actor loss:	-0.130150
critic loss:	4.239330
Episode 0: Going Reward = 2511.4: Std = 0.0
actor loss:	-1.338538
critic loss:	1.132295
Episode 20: Going Reward = 2506.7: Std = 734.1
actor loss:	0.126054
critic loss:	2.438496
Episode 40: Going Reward = 2570.2: Std = 282.6
actor loss:	0.462698
critic loss:	3.672793
Episode 60: Going Reward = 2617.9: Std = 259.9
actor loss:	-0.058580
critic loss:	3.679821
Episode 80: Going Reward = 2750.3: Std = 128.3
actor loss:	0.847187
critic loss:	3.922019
Episode 100: Going Reward = 2577.9: Std = 672.5
actor loss:	-0.207356
critic loss:	3.804548
Episode 120: Going Reward = 2787.8: Std = 94.0
actor loss:	0.024721
critic loss:	3.713242
Episode 140: Going Reward = 2749.5: Std = 118.5
actor loss:	0.026045
critic loss:	3.760216
Episode 160: Going Reward = 2707.0: Std = 121.6
actor loss:	0.100478
critic loss:	3.684100
Episode 180: Going Reward = 2674.1: Std = 657.1
actor loss:	-0.273155
critic loss:	2.782137
Episode 200: Going Reward = 2743.6: Std = 136.0
act

In [21]:
num_episodes = 1000
moving_rewards = np.array([])
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    # write the training loop
    done = False
    while not done:
        action = agent.select_action(state)
        next_state, reward, done, truncated, _ = env.step(action)
        agent.memory.rewards.append(reward)
        total_reward += reward
        state = next_state
        if done or truncated:
            break
    if episode % 5 == 0:
        loss_ac, loss_cri = agent.update()
        actor_losses.append(loss_ac)
        critic_losses.append(loss_cri)
    moving_rewards = np.append(moving_rewards, total_reward)
    if episode % 20 == 0:
        print(f"actor loss:\t{loss_ac:.6f}")
        print(f"critic loss:\t{loss_cri:.6f}")
        print(f'Episode {episode}: Going Reward = {moving_rewards.mean():.1f}: Std = {moving_rewards.std():.1f}')
        moving_rewards = np.array([])
env.close()

actor loss:	0.168839
critic loss:	3.911241
Episode 0: Going Reward = 3318.6: Std = 0.0
actor loss:	0.226204
critic loss:	3.651455
Episode 20: Going Reward = 3142.6: Std = 437.5
actor loss:	-0.088387
critic loss:	3.520103
Episode 40: Going Reward = 3283.5: Std = 163.7
actor loss:	0.059876
critic loss:	3.668634
Episode 60: Going Reward = 3274.6: Std = 120.4
actor loss:	-0.054896
critic loss:	3.573776
Episode 80: Going Reward = 3322.6: Std = 133.8
actor loss:	0.021216
critic loss:	3.781459
Episode 100: Going Reward = 3301.3: Std = 113.3
actor loss:	0.851911
critic loss:	3.853709
Episode 120: Going Reward = 3043.0: Std = 881.4
actor loss:	-0.776143
critic loss:	1.080667
Episode 140: Going Reward = 2654.6: Std = 1267.1
actor loss:	-0.386816
critic loss:	3.399240
Episode 160: Going Reward = 3122.3: Std = 225.6
actor loss:	0.508185
critic loss:	3.769274
Episode 180: Going Reward = 3080.7: Std = 266.0
actor loss:	-0.003045
critic loss:	3.437509
Episode 200: Going Reward = 3255.5: Std = 169.4
a

In [12]:
import torch
import json

def save_model_to_json(model, file_path):
    model_dict = {}
    for name, param in model.named_parameters():
        model_dict[name] = param.detach().cpu().numpy().tolist()
    with open(file_path, 'w') as f:
        json.dump(model_dict, f)

def load_model_from_json(model, file_path):
    with open(file_path, 'r') as f:
        model_dict = json.load(f)
    for name, weights in model_dict.items():
        model.parameters()[name].data = torch.Tensor(weights)

In [22]:
save_model_to_json(agent.actor, 'actor_model.json')
save_model_to_json(agent.critic, 'critic_model.json')

In [ ]:
load_model_from_json(agent.actor, 'actor_model.json')
load_model_from_json(agent.critic, 'critic_model.json')

## Evaluation **(5 points)**

Evaluate the trained policy on the environment. Calculate the cumulative reward and display the video of the trajectory.

In [23]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# run the policy in the environment in a loop
while True:
    with torch.no_grad():
        action = agent.select_action(state)
    state, reward, done, truncated, info = env.step(action)
    frames.append(env.render())
    total_reward += reward

    if done or truncated:
        break

env.close()
print(f'Total Reward: {total_reward}')
agent.memory.clear()
imageio.mimsave('./eval_ppo.mp4', frames, fps=25)
show_video('./eval_ppo.mp4')

Total Reward: 3504.277364653952
